In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LinearRegression
import random as rd 
import datetime 
import matplotlib.pyplot as plt 
import seaborn as sns
from math import ceil
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error



In [3]:
sales=pd.read_csv("/content/drive/MyDrive/competitive_data_science_predict_future_sales/sales_train.csv")
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
item_categories=pd.read_csv("/content/drive/MyDrive/competitive_data_science_predict_future_sales/item_categories.csv")
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [5]:
items=pd.read_csv("/content/drive/MyDrive/competitive_data_science_predict_future_sales/items.csv")
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [6]:
shops=pd.read_csv("/content/drive/MyDrive/competitive_data_science_predict_future_sales/shops.csv")
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [7]:
test_1=pd.read_csv("/content/drive/MyDrive/competitive_data_science_predict_future_sales/test.csv")
test_1.head()


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


# Outlier Removal

In [8]:
sales=sales[sales.item_price>0]
sales=sales[sales.item_price<43000]
sales=sales[sales.item_cnt_day>=0]
sales=sales[sales.item_cnt_day<550]


In [9]:
sales.isnull().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [10]:
sales.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.928483e+06,2.928483e+06,2.928483e+06,2.928483e+06,2.928483e+06
mean,1.456974e+01,3.300300e+01,1.020028e+04,8.893079e+02,1.246599e+00
std,9.422942e+00,1.622542e+01,6.324400e+03,1.717330e+03,2.119973e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e-02,1.000000e+00
25%,7.000000e+00,2.200000e+01,4.477000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.355000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.569100e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,4.299000e+04,5.390000e+02


In [11]:
sales=sales.merge(items[["item_id","item_category_id"]],on="item_id")
#sales.drop(["date"],axis=1,inplace=True)
sales=sales.sort_values('date').groupby(["date","date_block_num","shop_id","item_category_id","item_id"],as_index=False)
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
42989,01.01.2013,0,41,3402,549.0,1.0,30
1192079,01.01.2013,0,54,12864,248.0,1.0,40
83746,01.01.2013,0,2,1472,2599.0,1.0,23
847184,01.01.2013,0,54,10715,399.0,1.0,40
847518,01.01.2013,0,27,10450,349.0,1.0,40
...,...,...,...,...,...,...,...
2130328,31.12.2014,23,3,5660,1690.0,1.0,3
2130337,31.12.2014,23,56,5660,1690.0,1.0,3
2130339,31.12.2014,23,24,5660,1690.0,1.0,3
2130352,31.12.2014,23,21,5660,1690.0,1.0,3


In [12]:
sales=sales.agg({'item_price':['sum', 'mean'], 'item_cnt_day':['sum', 'mean']})
sales.head()

date date_block_num shop_id item_category_id item_id item_price  \
                                                                     sum   
0  01.01.2013              0       2                2    5649     2190.0   
1  01.01.2013              0       2               11   13434    13489.5   
2  01.01.2013              0       2               19    3320     1999.0   
3  01.01.2013              0       2               19    4464      599.0   
4  01.01.2013              0       2               19    6916      999.5   

           item_cnt_day       
      mean          sum mean  
0   2190.0          1.0  1.0  
1  13489.5          1.0  1.0  
2   1999.0          1.0  1.0  
3    599.0          1.0  1.0  
4    999.5          1.0  1.0

In [13]:
sales.columns = ["date",'date_block_num', 'shop_id', 'item_category_id', 'item_id','item_price', 'mean_item_price','item_cnt_monthly','item_cnt_mean']

In [14]:
month=[]
year=[]
month.append(sales["date"].apply(lambda x:int(x.split(".")[1])))
year.append(sales["date"].apply(lambda x:int(x.split(".")[2])))

In [15]:
sales["month"]=month[0]
sales["year"]=year[0]

In [16]:
sales.head()

,date,date_block_num,shop_id,item_category_id,item_id,item_price,mean_item_price,item_cnt_monthly,item_cnt_mean,month,year
0,01.01.2013,0,2,2,5649,2190.0,2190.0,1.0,1.0,1,2013
1,01.01.2013,0,2,11,13434,13489.5,13489.5,1.0,1.0,1,2013
2,01.01.2013,0,2,19,3320,1999.0,1999.0,1.0,1.0,1,2013
3,01.01.2013,0,2,19,4464,599.0,599.0,1.0,1.0,1,2013
4,01.01.2013,0,2,19,6916,999.5,999.5,1.0,1.0,1,2013


In [17]:
sales=sales.drop(["date"],axis=1)

# Rolling window

In [18]:
sales["rolling_max"]=sales.sort_values("date_block_num").groupby(["shop_id","item_category_id","item_id"])["item_cnt_monthly"].apply(lambda x:x.rolling(window=3, min_periods=1).max())
sales["rolling_min"]=sales.sort_values("date_block_num").groupby(["shop_id","item_category_id","item_id"])["item_cnt_monthly"].apply(lambda x:x.rolling(window=3, min_periods=1).min())
sales["rolling_mean"]=sales.sort_values("date_block_num").groupby(["shop_id","item_category_id","item_id"])["item_cnt_monthly"].apply(lambda x:x.rolling(window=3, min_periods=1).mean())
sales["rolling_std"]=sales.sort_values("date_block_num").groupby(["shop_id","item_category_id","item_id"])["item_cnt_monthly"].apply(lambda x:x.rolling(window=3, min_periods=1).std())

# lag features

In [20]:
sales["itm_cnt_lag_1"]=sales.groupby(["date_block_num","shop_id","item_category_id","item_id"])["item_cnt_monthly"].shift(1)
sales["itm_cnt_lag_13"]=sales.groupby(["date_block_num","shop_id","item_category_id","item_id"])["item_cnt_monthly"].shift(13)
sales["itm_cnt_lag_2"]=sales.groupby(["date_block_num","shop_id","item_category_id","item_id"])["item_cnt_monthly"].shift(2)

In [21]:
sales.bfill(axis ='rows')


,date_block_num,shop_id,item_category_id,item_id,item_price,mean_item_price,item_cnt_monthly,item_cnt_mean,month,year,rolling_max,rolling_min,rolling_mean,rolling_std,itm_cnt_lag_1,itm_cnt_lag_13,itm_cnt_lag_2
0,0,2,2,5649,2190.0,2190.0,1.0,1.0,1,2013,1.0,1.0,1.000000,0.000000e+00,1.0,2.0,1.0
1,0,2,11,13434,13489.5,13489.5,1.0,1.0,1,2013,1.0,1.0,1.000000,0.000000e+00,1.0,2.0,1.0
2,0,2,19,3320,1999.0,1999.0,1.0,1.0,1,2013,1.0,1.0,1.000000,0.000000e+00,1.0,2.0,1.0
3,0,2,19,4464,599.0,599.0,1.0,1.0,1,2013,1.0,1.0,1.000000,0.000000e+00,1.0,2.0,1.0
4,0,2,19,6916,999.5,999.5,1.0,1.0,1,2013,1.0,1.0,1.000000,0.000000e+00,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2928452,23,59,72,20610,1099.0,1099.0,1.0,1.0,12,2014,1.0,1.0,1.000000,0.000000e+00,1.0,1.0,1.0
2928453,23,59,75,4181,1190.0,1190.0,3.0,3.0,12,2014,3.0,1.0,1.666667,1.154701e+00,2.0,1.0,1.0
2928454,23,59,75,4186,1990.0,1990.0,1.0,1.0,12,2014,1.0,1.0,1.000000,0.000000e+00,4.0,1.0,1.0
2928455,23,59,79,17717,299.0,299.0,1.0,1.0,12,2014,1.0,1.0,1.000000,7.849674e-08,4.0,1.0,1.0


In [22]:
sales.fillna(0, inplace=True)

# Trend

In [37]:
sales["item_trend"]=sales["item_cnt_monthly"]
sales["item_trend"]=sales["item_trend"]-sales["itm_cnt_lag_1"]

In [38]:
sales["item_trend"]=sales["item_trend"]-sales["itm_cnt_lag_2"]


# Change in item price

In [25]:
min_max=sales.sort_values("date_block_num").groupby(["item_id"],as_index=False).agg({"item_price":[np.min, np.max]})
min_max.columns=["item_id","item_price_min","item_price_max"]
sales=pd.merge(sales,min_max,on="item_id",how='left')

sales["price_increase"]=sales["item_price"]-sales["item_price_min"]
sales["price_decrease"]=sales["item_price_max"]-sales["item_price"]

# Total sales in each shop

In [26]:
monthly_sales_per_shop=pd.DataFrame(sales.groupby(["shop_id"],as_index=False)["item_cnt_monthly"].sum())
monthly_sales_per_shop.rename(columns = {'item_cnt_monthly':'Total_sales'}, inplace = True)
sales["Total_sales_in_shop"]=monthly_sales_per_shop["Total_sales"]

# Total sales in each category

In [27]:
monthly_sales_item_cat=pd.DataFrame(sales.groupby(["item_category_id"],as_index=False)["item_cnt_monthly"].sum())
monthly_sales_item_cat.rename(columns = {'item_cnt_monthly':'Total_sales'}, inplace = True)
sales["Total_sales_item_cat"]=monthly_sales_item_cat["Total_sales"]

# Number of items in each category

In [28]:
item_count_item_cat=pd.DataFrame(sales.groupby(["item_category_id"],as_index=False)["item_id"].count())
item_count_item_cat.rename(columns = {'item_id':'number_of_item'}, inplace = True)
sales["item_cnt_iten_cat"]=item_count_item_cat["number_of_item"]

# Number of categories in each shop

In [29]:
item_cat_count_shops=pd.DataFrame(sales.groupby(["shop_id"],as_index=False)["item_category_id"].count())
item_cat_count_shops.rename(columns = {'item_category_id':'number_of_item_category'}, inplace = True)
sales["number_of_item_category"]=item_cat_count_shops["number_of_item_category"]

# Train Test split

In [39]:
train=sales[(sales.date_block_num>3)&(sales.date_block_num<33)]
valid=sales[(sales.date_block_num==33)]


**Mean encoding**

In [40]:
shop_encoded_mean_t=pd.DataFrame(train.groupby(["shop_id"])["item_cnt_monthly"].mean())
shop_encoded_mean_t.columns=["shop_mean"]
shop_encoded_mean_t.reset_index(inplace=True)

item_encoded_mean_t=pd.DataFrame(train.groupby(["item_id"])["item_cnt_monthly"].mean())
item_encoded_mean_t.columns=["item_mean"]
item_encoded_mean_t.reset_index(inplace=True)

item_cat_encoded_mean_t=pd.DataFrame(train.groupby(["item_category_id"])["item_cnt_monthly"].mean())
item_cat_encoded_mean_t.columns=["item_cat_mean"]
item_cat_encoded_mean_t.reset_index(inplace=True)

shop_item_encoded_mean_t=pd.DataFrame(train.groupby(["shop_id","item_id"])["item_cnt_monthly"].mean())
shop_item_encoded_mean_t.columns=["shop_item_mean"]
shop_item_encoded_mean_t.reset_index(inplace=True)

month_encoded_mean_t=pd.DataFrame(train.groupby(["month"])["item_cnt_monthly"].mean())
month_encoded_mean_t.columns=["month_mean"]
month_encoded_mean_t.reset_index(inplace=True)

year_encoded_mean_t=pd.DataFrame(train.groupby(["year"])["item_cnt_monthly"].mean())
year_encoded_mean_t.columns=["year_mean"]
year_encoded_mean_t.reset_index(inplace=True)

shop_encoded_mean_v=pd.DataFrame(valid.groupby(["shop_id"])["item_cnt_monthly"].mean())
shop_encoded_mean_v.columns=["shop_mean"]
shop_encoded_mean_v.reset_index(inplace=True)

item_encoded_mean_v=pd.DataFrame(valid.groupby(["item_id"])["item_cnt_monthly"].mean())
item_encoded_mean_v.columns=["item_mean"]
item_encoded_mean_v.reset_index(inplace=True)

item_cat_encoded_mean_v=pd.DataFrame(valid.groupby(["item_category_id"])["item_cnt_monthly"].mean())
item_cat_encoded_mean_v.columns=["item_cat_mean"]
item_cat_encoded_mean_v.reset_index(inplace=True)

shop_item_encoded_mean_v=pd.DataFrame(valid.groupby(["shop_id","item_id"])["item_cnt_monthly"].mean())
shop_item_encoded_mean_v.columns=["shop_item_mean"]
shop_item_encoded_mean_v.reset_index(inplace=True)

month_encoded_mean_v=pd.DataFrame(valid.groupby(["month"])["item_cnt_monthly"].mean())
month_encoded_mean_v.columns=["month_mean"]
month_encoded_mean_v.reset_index(inplace=True)

year_encoded_mean_v=pd.DataFrame(valid.groupby(["year"])["item_cnt_monthly"].mean())
year_encoded_mean_v.columns=["year_mean"]
year_encoded_mean_v.reset_index(inplace=True)





In [41]:
train=pd.merge(train,shop_encoded_mean_t, on='shop_id',how='left')
train=pd.merge(train,item_encoded_mean_t, on='item_id',how='left')
train=pd.merge(train,month_encoded_mean_t, on='month',how='left')
train=pd.merge(train,year_encoded_mean_t, on='year',how='left')
train=pd.merge(train,shop_item_encoded_mean_t, on=['shop_id','item_id'],how='left')

valid=pd.merge(valid,shop_encoded_mean_v, on='shop_id',how='left')
valid=pd.merge(valid,item_encoded_mean_v, on='item_id',how='left')
valid=pd.merge(valid,month_encoded_mean_v, on='month',how='left')
valid=pd.merge(valid,year_encoded_mean_v, on='year',how='left')
valid=pd.merge(valid,shop_item_encoded_mean_v, on=['shop_id','item_id'],how='left')


In [42]:
train.fillna(0, inplace=True)
valid.fillna(0, inplace=True)

In [43]:
train.head()

,date_block_num,shop_id,item_category_id,item_id,item_price,mean_item_price,item_cnt_monthly,item_cnt_mean,month,year,...,price_decrease,Total_sales_in_shop,Total_sales_item_cat,item_cnt_iten_cat,number_of_item_category,shop_mean,item_mean,month_mean,year_mean,shop_item_mean
0,12,2,2,5642,2490.0,2490.0,1.0,1.0,1,2014,...,800.0,0.0,0.0,0.0,0.0,1.198952,1.068365,1.215458,1.256738,1.000000
1,12,2,2,13508,2699.0,2699.0,1.0,1.0,1,2014,...,0.0,0.0,0.0,0.0,0.0,1.198952,1.000000,1.215458,1.256738,1.000000
2,12,2,3,5663,2599.0,2599.0,1.0,1.0,1,2014,...,1391.0,0.0,0.0,0.0,0.0,1.198952,1.292991,1.215458,1.256738,1.176471
3,12,2,11,13488,10691.0,10691.0,1.0,1.0,1,2014,...,3299.0,0.0,0.0,0.0,0.0,1.198952,1.196154,1.215458,1.256738,1.125000
4,12,2,19,1407,1299.0,1299.0,1.0,1.0,1,2014,...,1200.0,0.0,0.0,0.0,0.0,1.198952,1.360176,1.215458,1.256738,1.357143
